In [ ]:
"""
Python has 3 built-in functions that are designed to decorate methods
- property
- classmethod
- staticmethod

From functools module 3 more are interesting:
- wraps
- lru_cache
- singledispatch
"""

In [ ]:
# Memoization with functools.lru_cache
# Saves the results of previous invocations of an expensive function
# LRU - stands for Least Recently Used
# Growth of the cache is limited by discarding the entries that have
# not been read for a while

In [5]:
import time
import functools


def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [6]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

In [7]:
print(fibonacci(6))

[0.00000262s] fibonacci(1) -> 1 
[0.00000525s] fibonacci(0) -> 0 
[0.00157738s] fibonacci(2) -> 1 
[0.00000763s] fibonacci(1) -> 1 
[0.00755763s] fibonacci(3) -> 2 
[0.00000358s] fibonacci(1) -> 1 
[0.00005603s] fibonacci(0) -> 0 
[0.00224757s] fibonacci(2) -> 1 
[0.01188540s] fibonacci(4) -> 3 
[0.00000358s] fibonacci(1) -> 1 
[0.00000453s] fibonacci(0) -> 0 
[0.00043416s] fibonacci(2) -> 1 
[0.00000453s] fibonacci(1) -> 1 
[0.00077438s] fibonacci(3) -> 2 
[0.01379681s] fibonacci(5) -> 5 
[0.00000334s] fibonacci(1) -> 1 
[0.00000620s] fibonacci(0) -> 0 
[0.00381136s] fibonacci(2) -> 1 
[0.00000501s] fibonacci(1) -> 1 
[0.00429845s] fibonacci(3) -> 2 
[0.00000310s] fibonacci(1) -> 1 
[0.00000381s] fibonacci(0) -> 0 
[0.00033283s] fibonacci(2) -> 1 
[0.00496268s] fibonacci(4) -> 3 
[0.01912785s] fibonacci(6) -> 8 
8


In [8]:
@functools.lru_cache()  # lru_cache is invoked as func because it accepts parameters
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

In [9]:
print(fibonacci(6))

[0.00000334s] fibonacci(1) -> 1 
[0.00000548s] fibonacci(0) -> 0 
[0.00126553s] fibonacci(2) -> 1 
[0.00148106s] fibonacci(3) -> 2 
[0.00169802s] fibonacci(4) -> 3 
[0.00204825s] fibonacci(5) -> 5 
[0.00236630s] fibonacci(6) -> 8 
8


In [ ]:
# Besides making silly recursive algorithms viable lru shines in
# web apps
# It takes two arguments maxsize which is the size of cache
# and should be a power of 2
# and typed if set to true it distinguishes between int 1 and 
# float 1.0


In [10]:
import html


def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

In [ ]:
"""We would like it to behave differently depending on the 
type of the argument passed:
    >>> htmlize({1, 2, 3})
    '<pre>{1, 2, 3}</pre>'
    >>> htmlize(abs)
    '<pre>&lt;built-in function abs&gt;</pre>'
    >>> htmlize('Heimlich & Co.\n- a game')
    '<p>Heimlich &amp; Co.<br>\n- a game</p>'
    >>> htmlize(42)
    '<pre>42 (0x2a)</pre>'
    >>> print(htmlize(['alpha', 66, {3, 2, 1}]))

Common solution is to turn htmlize into a dispatch function
with a chain of if/elif/elif calling specialized functions
like htmlize_str. This isn't a good design it violates the
O in SOLID. Users of your code can't easily extend it, dispatcher
over time would grow really big and the coupling between specialized
functions would b every tight.

The functools.singledespatch decorator allows each module to
contribute to overall solution, and lets you easily provide a 
specialized function even for classes that you can't edit. If
you decorate a plain function with @singledispatch it becomes
generic function: a group of functions to perform the same
operation in different ways, depending on the type of the first 
argument. (If more arguments would to be used it would be called
 multiple-dispatch)"""


In [11]:
from functools import singledispatch
from collections import abc
import numbers
import html


@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'


@htmlize.register(str)
def _(text):  # The name of the registered function is irrelevant
    content = html.escape(text).replace('\n', '<br>\n')
    return f'<pre>{content}</pre>'


@htmlize.register(numbers.Integral)
def _(n):
    return f'<pre>{n} (0x{n:x})</pre>'


@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return f'<ul>\n<li>{inner}</li>\n</ul>'

In [ ]:
# When possible prefer abstract classes instead of concrete
# implementations like int or list. For example python 
# extensions provide alternatives to the int type with fixed bit
# lengths as subclasses of number.Integral

In [ ]:
# You can stack decorators which is the same as:
f = d1(d2(f))

@d1
@d2
def f():
    pass